- I'll rapidly go over the changes i made to the agent and environment to accomodate the DQN approach
    - model taken from <a href="https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html">pytorch's site<a/>
    - the invalid action cost in the environment is now set to -0.01, for the following reason: in order to follow the convention to choose the maximum action-value, in the TSP context this would convert to the 1/reward (where the reward is the distance between nodes). A bigger distance results to a lower action value.

In [ ]:
import torch as torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from deep_q.custom_env_deep_q import TSPEnv


class TSPDistCost(TSPEnv):
    # (...)

    def __init__(self, *args, **kwargs):
        self.N = 7

        self.invalid_action_cost = -0.01
    # (...)

- This also happens in the step process, where the reward is again defined as the inverse

In [ ]:
    def _STEP(self, action):
        done = False

        #check if the action chosen is the starting one
        #this next condition should be implemented for an environment that intrinsecally requires
        # a closed loop, not the case for the or-gym context
        #if action == self.start_node and self.step_count<(self.N + 1):


        if self.visit_log[action] > 0:
            # Node already visited
            #changed for DQN
            reward = 1/self.invalid_action_cost
            done = True
        else:
            #changed for DQN
            if(self.current_node == action):
                reward = 0
            else:
                reward = 1 / self.distance_matrix[self.current_node, action]
            self.current_node = action
            self.visit_log[self.current_node] = 1

        self.state = self._update_state()
        # See if all nodes have been visited
        unique_visits = self.visit_log.sum()

        if unique_visits == self.N:
            done = True

        return self.state, reward, done, {}

# DQN
- The DQN is defined as follows, please tell me if it makes sense
    - it takes as input the state and outputs a tensor of action values, to determine the best action to take (in theory the one with the highest value). The index of the action_value corresponds to the action (node chosen) to make.
    - the initial epsilon is 0.2 here, but i changed it around up to 0.5 to see the effects on the learning process. 0.2 seemed to be optimal.
    - I tried ReLu, tanh, softmax and sigmoid just to see the effects, for now RReLu seemed to be the best performing activation function. I'll try more combinations later on, with a more rational approach and not just by trial and error.
    - The minibatch size at the beginning was 64, but the learning didnt look to be going at a satisfying rate. I changed it to 16, but I suppose i'll have to push it back to 64 as the number of nodes gets higher.

In [ ]:
from torch import nn

class DQN(nn.Module):
    def __init__(self, n_nodes):
        super(DQN, self).__init__()

        self.n_actions = n_nodes
        self.gamma = 0.99
        self.final_epsilon = 0.0001
        self.initial_epsilon = 0.2
        self.decay_epsilon = 7000
        self.num_iterations = 2000000
        self.replay_mem_size = 10000
        self.minibatch_size = 16
        #inplace=True means that it will modify the input directly, without allocating any additional output.
        self.relu1 = nn.RReLU(inplace=True)

        self.relu2 = nn.RReLU(inplace=True)

        self.relu3 = nn.RReLU(inplace=True)
        self.fc4   = nn.Linear(in_features=8, out_features=512)
        self.relu4 = nn.RReLU(inplace = True)
        self.fc5 = nn.Linear(in_features=512, out_features=n_nodes)

    def forward(self, x):
        floats = []
        for val in x:
            floats.append(val.double())
        #then convert from list to tensor
        tensFloats = torch.stack(floats)
        out = self.relu1(tensFloats)
        out = self.relu2(out)
        out = self.relu3(out)
        out = (out.float())
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)

        return out

# Agent
The agent is defined as follows
- it has a policy and a target network. The policy net determines the actions ot be taken by the agent, it's fed the current state of the environment. I'll have to read up more in depth about the actual usefulness of the target network, I understood it's for stability purposes, but I have yet to understand where and when to put it exactly for future applications.
- I thought a replay buffer of 10k experiences (s, a, s', r tuples) was enough for this model, will see if it needs to be exponentially scaled up for bigger applications.
- the Optimization algorithm is Adam, have not yet played with other algorithms.
(some of the lines here are purely for my future reference for writing down the actual thesis)
- It combines the advantages of
   - Adaptive Gradient Algorithm :
       - Adaptive Gradient Algorithm (AdaGrad) that maintains a per-parameter learning rate that improves performance on problems with sparse gradients (e.g. natural language and computer vision problems).
   - RMS propagation
        - Root Mean Square Propagation, that also maintains per-parameter learning rates that are adapted based on the average of recent magnitudes of the gradients for the weight (e.g. how quickly it is changing). This means the algorithm does well on online and non-stationary problems (e.g. noisy).
- Huber Loss for loss calculation
    - ref here <a href="https://en.wikipedia.org/wiki/Huber_loss"> wiki page</a>

## Action Selection
- epsilon greedy policy, will start with the value defined in the agent initialization and will decade over time to reduce the number of random actions taken.
- ```python
  index_action = torch.argmax(self.policy_net(state))
  ```
  will pass the state through the NN, estimate the action values and choose the index of the highest action value, which will determine the best action at this point in time.
- the random choice will be taken with
- ```python
    torch.tensor([[random.randint(0,self.env.N -1 )]], device=device, dtype=torch.long)
```

In [ ]:
 import random


def select_action(self, state):
        global steps_done
        sample = random.random()
        eps_threshold = self.policy_net.final_epsilon + (self.policy_net.initial_epsilon - self.policy_net.final_epsilon) * math.exp(-1. * steps_done / self.policy_net.decay_epsilon)
        steps_done += 1
        # if(steps_done%400 == 0):
        #     print(f"{steps_done} --> eps : {eps_threshold}" )
        if sample > eps_threshold:
            with torch.no_grad():
                # this will extract the index of the action_value with the current highest
                #value, ensuring that the action with the highest possible (expected) reward is chosen.
                index_action = torch.argmax(self.policy_net(state))
                return index_action

        else:
            return torch.tensor([[random.randint(0,self.env.N -1 )]], device=device, dtype=torch.long)

### Model Optimization
- Probably the most important function, every batch_size steps it extracts a batch of experiences

In [ ]:
    def optimize_model(self):
        if len(self.exp_buffer) < self.policy_net.minibatch_size:
            return
        transitions = self.exp_buffer.sample(self.policy_net.minibatch_size)
        # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
        # detailed explanation).
        # This converts batch-array of Experiences to Experience of batch-arrays.

        batch = Experience(*zip(transitions))



Non final states are masked as False, Final states (None values) are True

In [ ]:

        # Compute a mask of non-final states and concatenate the batch elements
        # (a final state would've been the one after which simulation ended)
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.new_state[0])), device=device, dtype=torch.bool)


Extract non final states, will use a more efficient solution as soon as i am sure about the whole algorithm.

In [ ]:

        tmp_non_final_next_states = []
        for val in batch.new_state[0]:
            if val is not None:
                tmp_non_final_next_states.append(val)

        #shape is (batch_size, num_nodes + 1)
        non_final_next_states = torch.stack(tmp_non_final_next_states)

        #concatenate non final states in the batch in non_final_next_states
        state_batch = torch.stack(batch.state[0])

        #reshaping to ensure same dimensions as state_action_values
        action_batch_reshape = (np.reshape(batch.action[0], (self.policy_net.minibatch_size,1))).astype(np.int64)
        action_batch_v2 = torch.as_tensor(action_batch_reshape)

        reward_batch = torch.from_numpy(batch.reward[0])

        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
        # columns of actions taken. These are the actions which would've been taken
        # for each batch state according to policy_net



The next line will first feed the batch of states to the network, which will output the action values of the states.
To make this a bit clearer, let's take for example a batch of size 2
    - [0] = (6,0,0,0,1,0,1,1)
    - [1] = (3,0,1,1,1,0,0,0)
will be fed to the NN, which will output two tensors of size (n_actions), for example with these values
    - [0]     (0.34, 0.79, -0.55, 0.22, 0.11, 0.59)
    - [1]     (0.89, -0.98, 0.34, 0.84, 0.32, -0.11)

the action_batch will be for example [3, 2] (as these were the actions taken) in the experiences. The state_action values extracted will be at index 3 for row 0, 2 for row 1, so [0.22, 0.34]


In [ ]:
        state_action_values = self.policy_net(state_batch).gather(1, action_batch_v2)


        # Compute V(s_{t+1}) for all next states.
        # Expected values of actions for non_final_next_states are computed based
        # on the "older" target_net; selecting their best reward with max(1)[0].
        # This is merged based on the mask, such that we'll have either the expected
        # state value or 0 in case the state was final.
        next_state_values = torch.zeros(self.policy_net.minibatch_size, device=device)

        tmp_next_state_values = self.target_net(non_final_next_states)


Extract max state action values from the next states

In [ ]:
        max, ind =  torch.max(tmp_next_state_values,dim=1)
        next_state_values[non_final_mask] = max
        # Compute the expected Q values
        reward_batch = torch.FloatTensor(reward_batch)
        expected_state_action_values = (next_state_values * self.policy_net.gamma) + reward_batch

        # Compute Huber loss
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
        #print(loss)

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        #loss returned purely to see the progress made by the model

        return loss

### Episode being played

In [ ]:
    from deep_q.replay_memory import Experience


def play_episodes(self):
        def display_distances(matrix, path):
            print("Distances in path :")
            tot_dist = 0
            for _ in range(1, len(path)):
                dist = matrix[path[_]][path[_ - 1]]
                tot_dist += dist
                print(f"{path[_ - 1]} --> {path[_]} : {dist}")
            print(f"Total distance : {tot_dist}")

        num_episodes = 20000
        for i_episode in range(num_episodes):
            tot_reward = 0
            # Initialize the environment and state
            self.env.reset()
            path = [self.env.state[0].item()]


            state = self.env.state
            for t in range(self.env.step_limit):#count():
                # Select and perform an action
                action = self.select_action(state).item()
                _, reward, done, _ = self.env.step(action)
                path.append(action)
                #reward = torch.tensor(reward, device=device)
                tot_reward += reward

                # Observe new state

                if not done:
                    next_state = self.env.state
                else:
                    next_state = None#torch.ones([self.env.N + 1], dtype=torch.int32)*-1

                # Store the transition in memory
                self.exp_buffer.append(Experience(state, action,reward, done, next_state))

                # Move to the next state
                state = next_state

                # Perform one step of the optimization (on the policy network)
                loss = self.optimize_model()
                if done:

                    #close loop
                    path.append(path[0])
                    #print(f"path {path}: {reward}")

                    break
            # Update the target network, copying all weights and biases in DQN
            if i_episode % 10 == 0:
                self.target_net.load_state_dict(self.policy_net.state_dict())

            if i_episode % 512 == 0:
                print(f"path {path}: {tot_reward}, loss = {loss}")
                display_distances(self.env.distance_matrix, path)
                self.env.render_custom(path)


        print('Complete')
        #self.env.render_custom(path)

## Issues so far
- after loss goes under 0.4 / 0.3, the NN actually seems to take worse decisions, probably will set it as cut off point or will play around with hyperparameters. why does it behave like this?
- the way the environment is set, it doesnt really consider closing the loop. the way i implemented it so far is to calculate the fastest route to traverse all nodes and close the loop by rudimentally connecting last and first node. I assume this isn't exactly the best implementation for TSP, but the or-gym environment is set in the following way for the ending condition

In [ ]:
 unique_visits = self.visit_log.sum()
        if unique_visits == self.N:
            done = True

And when it resets the environment it sets the initial node to 1 (visited).

In [ ]:
self.visit_log[self.current_node] += 1

Unless im making a gross mistake this model terminates when all the nodes have been visited (and not for N+1, which would imply a closed path). I could think about another solution and change the current model if you require a closed loop to be considered in the state-action value operation.